In [2]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import time

import os, json, sys
sys.path.append("../../../../../resources/pyimgsaliency/")
import pyimgsaliency as psal

# test saliency detection
demo_img = "../../../../../resources/pyimgsaliency/pyimgsaliency/bird.jpg"
demo_img = "video-frames/equation-2/00006.png"
start_time = time.time()
rbd = psal.get_saliency_rbd(demo_img).astype('uint8')
after_time = time.time()
print "RBD", after_time - start_time

start_time = time.time()
ft = psal.get_saliency_ft(demo_img).astype('uint8')
after_time = time.time()
print "FT", after_time - start_time
import scipy.misc
scipy.misc.toimage(ft).save("/Users/apavel/Desktop/meow.png")
plt.imshow(ft)



start_time = time.time()
rbd = psal.get_saliency_mbd(demo_img).astype('uint8')
after_time = time.time()
print "MBD", after_time - start_time

IOError: [Errno 2] No such file or directory: 'video-frames/equation-2/00006.png'

In [6]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import time

import os, json, sys
sys.path.append("../../../../../resources/pyimgsaliency/")
import pyimgsaliency as psal

import scipy.misc, os
import subprocess
from PIL import Image

VIDEO_FRAMES_ROOT = "video-frames/"
ANALYSIS_ROOT = "analysis/image-saliency/"
VIDEO_ROOT = "videos/"

time_ref = {
    "snowboarding": [151.868109 + 29, 241],
    "hpo-preview": [0, 60.5],
    "surfing": [45, 105],
    "dining-at-the-met": [30, 96],
    "knives": [0,35],
    "trees": [82,150],
    "invasion": [120, 180],
    "ice-art": [4,70],
    "volcano": [93,160]
}
bases = time_ref.keys()

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def any_base_in_fn(f): 
        for base in bases: 
            if base in f: 
                return True
        return False

all_videos = [v for v in os.listdir(VIDEO_ROOT) if "small" in v and v[0] !='.' if "arctic" not in v and "congo" not in v]
all_videos = [v for v in all_videos if any_base_in_fn(v)]
print all_videos

for video in all_videos:
    base = video.lower().split(".mp4")[0]
    print "Processing video:", base
    video_path = VIDEO_ROOT + video
    video_frames_path = VIDEO_FRAMES_ROOT + base + "/"
    
    # delete any existing video frames
    sub_directory_paths = [VIDEO_FRAMES_ROOT + p + "/" for p in get_immediate_subdirectories(VIDEO_FRAMES_ROOT)]
    for d in sub_directory_paths: 
        for frame in os.listdir(d):
            os.remove(d + frame)
            
    # make a new directory
    if not os.path.exists(video_frames_path):
        os.mkdir(video_frames_path)
    
    start_time = time.time()
    
    # split the video into smaller video frames
    subprocess.call(["ffmpeg", "-i", video_path, "-vf", "fps=2", video_frames_path + "%05d.png"])
    
    analysis_path = ANALYSIS_ROOT + base + "/"
    
    # make a new directory
    if not os.path.exists(analysis_path):
        os.mkdir(analysis_path)
    
    # ok now we actually want to just take 2 frames per second
    fns = [video_frames_path + fn for fn in os.listdir(video_frames_path) if ".png" in fn]
#     new_fns = []
#     for i in range(len(fns)): 
#         fn = fns[i]
#         if i % 15 == 0: 
#             new_fns.append(fn)
#     fns = new_fns
    
    for frame_fn in fns: 
        out_fn = analysis_path + frame_fn.split("/")[-1]
        try: 
            sal_map = psal.get_saliency_mbd(frame_fn).astype('uint8')
            im = Image.fromarray(sal_map)
            im.save(out_fn)
        except: 
            print frame_fn
        
    print "Time to process:", time.time() - start_time  

['dining-at-the-met-small.mp4', 'hpo-preview-small.mp4', 'ice-art-no-text-small.mp4', 'ice-art-small.mp4', 'invasion-small.mp4', 'knives-long-small.mp4', 'knives-small.mp4', 'snowboarding-small.mp4', 'surfing-small.mp4', 'trees-small.mp4', 'volcano-small.mp4']
Processing video: dining-at-the-met-small
video-frames/dining-at-the-met-small/00001.png
video-frames/dining-at-the-met-small/00002.png
video-frames/dining-at-the-met-small/00003.png
video-frames/dining-at-the-met-small/00004.png
video-frames/dining-at-the-met-small/00005.png
video-frames/dining-at-the-met-small/00006.png
video-frames/dining-at-the-met-small/00007.png
video-frames/dining-at-the-met-small/00239.png
video-frames/dining-at-the-met-small/00240.png
video-frames/dining-at-the-met-small/00241.png
video-frames/dining-at-the-met-small/00242.png
video-frames/dining-at-the-met-small/00243.png
video-frames/dining-at-the-met-small/00244.png
video-frames/dining-at-the-met-small/00245.png
video-frames/dining-at-the-met-small/00

../../../../../resources/pyimgsaliency/pyimgsaliency/saliency_mbd.py:214: RuntimeWarning: invalid value encountered in maximum
  u_max = np.maximum(np.maximum(np.maximum(u_left,u_right),u_top),u_bottom)
../../../../../resources/pyimgsaliency/pyimgsaliency/saliency_mbd.py:220: RuntimeWarning: invalid value encountered in divide
  sal = sal / sal_max + u_final / u_max_final



video-frames/knives-long-small/00079.png
video-frames/knives-long-small/00080.png
video-frames/knives-long-small/00081.png
video-frames/knives-long-small/00082.png
video-frames/knives-long-small/00105.png
video-frames/knives-long-small/00106.png
video-frames/knives-long-small/00107.png
video-frames/knives-long-small/00108.png
Time to process: 119.472592831
Processing video: knives-small
video-frames/knives-small/00001.png
video-frames/knives-small/00017.png
video-frames/knives-small/00018.png
video-frames/knives-small/00028.png
Time to process: 74.3885400295
Processing video: snowboarding-small
video-frames/snowboarding-small/00001.png
video-frames/snowboarding-small/00002.png
video-frames/snowboarding-small/00003.png
video-frames/snowboarding-small/00004.png
video-frames/snowboarding-small/00005.png
video-frames/snowboarding-small/00006.png
video-frames/snowboarding-small/00007.png
video-frames/snowboarding-small/00008.png
video-frames/snowboarding-small/00009.png
video-frames/snowbo

##### rbd

In [ ]:
fti.